In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import os, time, math, json
with open('/Users/yawagipong/.secret/tmdb_api.json', 'r') as f:
          login = json.load(f)
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok = True)
os.listdir(FOLDER)

['title.ratings.csv.gz',
 'title_basics.csv.gz',
 '.ipynb_checkpoints',
 'title.akas.csv.gz']

In [2]:
tmdb.API_KEY = login['api-key']

In [3]:
def get_movie_with_rating(movie_id):
    
    movie = tmdb.Movies(movie_id)
    
    info = movie.info()
    
    releases = movie.releases()
    
    for c in releases['countries']:
        
        if c['iso_3166_1'] == 'US':
            
            info['certification'] = c['certification']
            
    return info

In [4]:
get_movie_with_rating("tt0848228")

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 122.502,
 'poster_path': '/tYqp6vEOo8YlVWrYQvt9nyOhsA2.jpg',
 'production_companies': [{'id': 420,
   'logo_pat

In [11]:
get_movie_with_rating("tt0332280")

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 64.762,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/iaYpEp3LQmb8AfAtmTvpqd4149c.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [5]:
def write_json(new_data, filename):
    
    with open(filename, 'r+') as file:
        
        file_data = json.load(file)
        
        if (type(new_data) == list) & (type(file_data) == list):
            
            file_data.extend(new_data)
            
        else:
            
            file_data.append(new_data)
            
        file.seek(0)
        
        json.dump(file_data, file)

In [6]:
def read_and_fix_json(JSON_FILE):
    """Attempts to read in json file of records and fixes the final character
    to end with a ] if it errors.
    
    Args:
        JSON_FILE (str): filepath of JSON file
        
    Returns:
        DataFrame: the corrected data from the bad json file
    """
    try: 
        previous_df =  pd.read_json(JSON_FILE)
    
    ## If read_json throws an error
    except:
        
        ## manually open the json file
        with open(JSON_FILE,'r+') as f:
            ## Read in the file as a STRING
            bad_json = f.read()
            
            ## if the final character doesn't match first, select the right bracket
            first_char = bad_json[0]
            final_brackets = {'[':']', 
                           "{":"}"}
            ## Select expected final brakcet
            final_char = final_brackets[first_char]
            
            ## if the last character in file doen't match the first char, add it
            if bad_json[-1] != final_char:
                good_json = bad_json[:-1]
                good_json+=final_char
            else:
                raise Exception('ERROR is not due to mismatched final bracket.')
            
            ## Rewind to start of file and write new good_json to disk
            f.seek(0)
            f.write(good_json)
           
        ## Load the json file again now that its fixed
        previous_df =  pd.read_json(JSON_FILE)
        
    return previous_df

In [7]:
basics = pd.read_csv('Data/title_basics.csv.gz')

In [8]:
YEARS_TO_GET = [2000, 2001]

In [9]:
errors = []

In [10]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc = 'YEARS', position = 0):
    
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == False:
        
        with open(JSON_FILE, 'w') as f:
            
            json.dump([{'imdb_id':0}], f)
            
    df = basics.loc[basics['startYear'] == YEAR].copy()
    
    movie_ids = df['tconst'].copy()
    
    previous_df = read_and_fix_json(JSON_FILE)
    
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                 desc = f'Movies from {YEAR}',
                                 position = 1,
                                 leave = True):
        
        try:
            
            temp = get_movie_with_rating(movie_id)
            
            write_json(temp, JSON_FILE)
            
            time.sleep(0.02)
            
        except Exception as e:
            
            errors.append([movie_id, e])
            
    final_year_df = pd.read_json(JSON_FILE)
    
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression ="gzip", index = False)
    
    
print(f" - Total errors: {len(errors)}")

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1428 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1541 [00:00<?, ?it/s]

 - Total errors: 455
